## 即時影像處理：
- 用 OpenCV 讀取攝影機畫面，進行即時的背景減除（MOG2 或 Frame Difference）。
- 在畫面中標註出移動的物體，並在畫面中為人臉或物體添加馬賽克處理。
## 移動偵測：
- 用背景減除或 Frame Difference 檢測影片中的移動物體。
- 為移動物體添加邊界框，並進行顯示或後續處理。

## 進階背景減除：
- 使用不同的背景減除算法來處理各種環境（如動態背景、陰影等）。

In [10]:
import cv2
import os
import glob

In [12]:
# 讀取影像序列
image_folder = "./cdnet2014/dataset/baseline/PETS2006/input"
image_paths = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))

In [16]:
# 使用 MOG2 背景減除器
fgbg_mog2 = cv2.createBackgroundSubtractorMOG2()

# 使用 KNN 背景減除器
fgbg_knn = cv2.createBackgroundSubtractorKNN()

for image_path in image_paths:
    # 讀取圖片
    frame = cv2.imread(image_path)

    # 背景減除：使用 MOG2
    fgmask_mog2 = fgbg_mog2.apply(frame)
    
    # 背景減除：使用 KNN
    fgmask_knn = fgbg_knn.apply(frame)
    
    # 顯示結果
    cv2.imshow('MOG2', fgmask_mog2)
    cv2.imshow('KNN', fgmask_knn)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [15]:
# 背景減除器（MOG2）
fgbg = cv2.createBackgroundSubtractorMOG2()

for image_path in image_paths:
    # 讀取圖片
    frame = cv2.imread(image_path)

    # 背景減除
    fgmask = fgbg.apply(frame)

    # 顯示背景減除效果（前景遮罩）
    cv2.imshow('Foreground Mask', fgmask)

    # 顯示原始畫面
    cv2.imshow('Original Frame', frame)

    # 偵測移動物體：用輪廓檢測標註移動物體
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # 過濾小輪廓
            x, y, w, h = cv2.boundingRect(contour)
            # 在移動物體周圍畫上紅色邊界框
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # 顯示標註了移動物體的原始畫面
    cv2.imshow('Detected Movement', frame)

    # 按 'q' 鍵退出
    if cv2.waitKey(30) & 0xFF == ord('q'):  # 每幀顯示 30 毫秒
        break
        
# 釋放資源
cv2.destroyAllWindows()